In [1]:
cd(@__DIR__) #makes the directory where this script is located the new working directory
using Pkg
Pkg.activate()
Pkg.instantiate()
using MixedModels
using RCall
using DataFrames
using CSV
using RData
using Statistics
using Dates

 Activating environment at `C:\Users\bjoer\.julia\environments\v1.4\Project.toml`
┌ Warning: RCall.jl: Warning: package 'dplyr' was built under R version 3.6.3
│ Warning: replacing previous import 'vctrs::data_frame' by 'tibble::data_frame' when loading 'dplyr'
│ 
│ Attaching package: 'dplyr'
│ 
│ The following objects are masked from 'package:stats':
│ 
│     filter, lag
│ 
│ The following objects are masked from 'package:base':
│ 
│     intersect, setdiff, setequal, union
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\AEOQ7\src\io.jl:160
┌ Warning: RCall.jl: Warning: package 'tidyverse' was built under R version 3.6.3
│ -- Attaching packages --------------------------------------- tidyverse 1.3.0 --
│ v ggplot2 3.3.2     v purrr   0.3.4
│ v tibble  3.0.3     v stringr 1.4.0
│ v tidyr   1.1.0     v forcats 0.5.0
│ v readr   1.3.1     
│ Warning: package 'ggplot2' was built under R version 3.6.3
│ Warning: package 'tibble' was built under R version 3.6.3
│ Warning: package 'tidyr' w

RObject{ClosSxp}
function (ID, ConditionOfInterest, StandardValues, reps, PSE_Difference, 
    JND_Difference, Multiplicator_PSE_Standard, Multiplicator_SD_Standard, 
    SD_ResponseFunction, Mean_Variability_Between = 0.2, SD_Variability_Between = 0.2) 
{
    Psychometric = expand.grid(ID = ID, ConditionOfInterest = ConditionOfInterest, 
        StandardValues = StandardValues, reps = reps)
    Psychometric = Psychometric %>% group_by(ID) %>% mutate(PSE_Factor_ID = rnorm(1, 
        1, Mean_Variability_Between), SD_Factor_ID = rnorm(1, 
        1, SD_Variability_Between))
    Psychometric = Psychometric %>% mutate(Mean_Standard = StandardValues + 
        StandardValues * Multiplicator_PSE_Standard, SD_Standard = StandardValues * 
        Multiplicator_SD_Standard, Mean = (Mean_Standard + (ConditionOfInterest == 
        ConditionOfInterest[2]) * StandardValues * PSE_Difference) * 
        PSE_Factor_ID, SD = abs((SD_Standard + (ConditionOfInterest == 
        ConditionOfInterest[2]) 

We first write a function that allows us to simulate psychophysical data. We describe this in more detail in the paper. We simulate binary yes-no responses to different stimulus strengths whose proportions should be well described by a cummulative Gaussian function.

In [ ]:
R"""
require(dplyr, quietly = TRUE)   # for data wrangling
require(tidyverse, quietly = TRUE)   # for data wrangling
require(lme4)
require(lmerTest)
require(quickpsy)


SimulatePsychometricFunction_Staircase = function(ID, 
                                                  ConditionOfInterest,
                                                  StandardValues,
                                                  reps,
                                                  PSE_Difference,
                                                  JND_Difference,
                                                  Multiplicator_PSE_Standard,
                                                  Multiplicator_SD_Standard,
                                                  SD_ResponseFunction,
                                                  Mean_Variability_Between = 0.2,
                                                  SD_Variability_Between = 0.2){
  Psychometric = expand.grid(ID=ID, ConditionOfInterest=ConditionOfInterest, StandardValues=StandardValues, reps = reps)
      
  Psychometric = Psychometric %>%
        group_by(ID) %>%
  mutate(PSE_Factor_ID = rnorm(1,1,Mean_Variability_Between),
               SD_Factor_ID = rnorm(1,1,SD_Variability_Between))
         
      Psychometric = Psychometric %>%
         mutate(
          Mean_Standard = StandardValues+StandardValues*Multiplicator_PSE_Standard,
                 SD_Standard = StandardValues*Multiplicator_SD_Standard,
          Mean = (Mean_Standard + (ConditionOfInterest==ConditionOfInterest[2])*StandardValues*PSE_Difference)*PSE_Factor_ID,
                 SD = abs((SD_Standard + (ConditionOfInterest==ConditionOfInterest[2])*SD_Standard*JND_Difference)*SD_Factor_ID),
          staircase_factor = rcauchy(length(reps),1,SD_ResponseFunction), 
                 Presented_TestStimulusStrength = Mean*staircase_factor,
          Difference = Presented_TestStimulusStrength - StandardValues,
                 AnswerProbability = pnorm(Presented_TestStimulusStrength,Mean,SD),
          Answer = as.numeric(rbernoulli(length(AnswerProbability),AnswerProbability))
         )
      
         Psychometric = Psychometric %>%
        filter(abs(staircase_factor-1) < 0.75) %>%
         group_by(ID,ConditionOfInterest,StandardValues,Difference) %>%
        mutate(Yes = sum(Answer==1),
         Total = length(ConditionOfInterest))
      
  Psychometric
    }
"""

We then choose values that are interesting to us. We go with two different StandardValues, 30-60 trials per staircase and a range of 10-20 participants. The other values would be somewhat appropriate for our example (speed perception). We choose no difference between the standard and the test conditions to evaluate false positives.

We then set up the different models we want to try (both for R and for Julia) and cycle through them with each combination of values chosen. For each, we save a couple of outcomes, like the approximated p value, the AIC and the fitting duration. Then we save all of thesr values for each iteration to an excel file.

In [2]:
#####Compare between Julia and R: no difference
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [30,40,50,60]
PSE_Difference = 0
JND_Difference = 0
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.2
SD_Variability_Between = 0.2
nIterations = 25
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()

formula1 = @formula(Answer ~ Difference*ConditionOfInterest + 
                            (Difference + ConditionOfInterest |ID) + 
                            (Difference|StandardValues));
formulaNull_Interaction = @formula(Answer ~ Difference + ConditionOfInterest + 
                            (Difference + ConditionOfInterest | ID) + 
                            (Difference | StandardValues));

for reps in Range_reps
    for n in Range_Participants
        CurrentRunthrough = CurrentRunthrough + 1

        if CurrentRunthrough > 1
        print("This runthrough took ", Dates.now()-TimeStartRunthrough, " s")                
        end
        
        print("Starting runthrough N° ", CurrentRunthrough, " out of ", TotalNumber)
        global TimeStartRunthrough = Dates.now()

        for j in 1:nIterations
            
            @rput reps n j ConditionOfInterest StandardValues PSE_Difference JND_Difference Multiplicator_PSE_Standard
            @rput Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between
            
            #simulate data in R with the function from above and fit models
            R"""
            ID = paste0("s",1:n)
            
            TimeStartTrial = Sys.time() #get time at beginning of trial            
            Psychometric = SimulatePsychometricFunction_Staircase(ID,
                            ConditionOfInterest,
                            StandardValues,
                            1:reps,
                            PSE_Difference,
                            JND_Difference,
                            Multiplicator_PSE_Standard,
                            Multiplicator_SD_Standard,
                            SD_ResponseFunction,
                            Mean_Variability_Between,
                            SD_Variability_Between)
            
              Psychometric$StandardValues = as.character(Psychometric$StandardValues)
              Duration_SimulateDataset = Sys.time() - TimeStartTrial #get duration of fitting
            
              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (ConditionOfInterest+Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 0)
              RDuration_NelderMead_nAGQ0 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_NelderMead_nAGQ0 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_NelderMead_nAGQ0 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_NelderMead_nAGQ0 = AIC(GLMM)
              RSEs_NelderMead_nAGQ0 = summary(GLMM)$coefficients[c(6,8)]
            
              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (ConditionOfInterest+Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 1)
              RDuration_NelderMead_nAGQ1 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_NelderMead_nAGQ1 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_NelderMead_nAGQ1 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_NelderMead_nAGQ1 = AIC(GLMM)
              RSEs_NelderMead_nAGQ1 = summary(GLMM)$coefficients[c(6,8)]
            
              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (ConditionOfInterest+Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 0,
                           glmerControl(optimizer = "bobyqa"))
              RDuration_Bobyqa_nAGQ0 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_Bobyqa_nAGQ0 = summary(GLMM)$coefficients[c(14,16)] 
              RCoefficients_Bobyqa_nAGQ0 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_Bobyqa_nAGQ0 = AIC(GLMM)
              RSEs_Bobyqa_nAGQ0 = summary(GLMM)$coefficients[c(6,8)]
            
              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (ConditionOfInterest+Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 1,
                           glmerControl(optimizer = "bobyqa"))
              RDuration_Bobyqa_nAGQ1 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_Bobyqa_nAGQ1 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_Bobyqa_nAGQ1 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_Bobyqa_nAGQ1 = AIC(GLMM)
              RSEs_Bobyqa_nAGQ1 = summary(GLMM)$coefficients[c(6,8)]
            
              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (ConditionOfInterest+Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 0,
                           glmerControl(optimizer = "nloptwrap"))
              RDuration_nloptwrap_nAGQ0 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_nloptwrap_nAGQ0 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_nloptwrap_nAGQ0 = summary(GLMM)$coefficients[c(2,4)]            
              RAIC_nloptwrap_nAGQ0 = AIC(GLMM)
              RSEs_nloptwrap_nAGQ0 = summary(GLMM)$coefficients[c(6,8)]            
            
              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMMNull = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest + Difference + (ConditionOfInterest+Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 0,
                           glmerControl(optimizer = "nloptwrap"))
              RPvalues_nloptwrap_nAGQ0_LRT = anova(GLMM,GLMMNull)$`Pr(>Chisq)`[2]
              RDuration_nloptwrap_nAGQ0_LRT = Sys.time() - TimeStartTrial #get duration of fitting            

              TimeStartTrial = Sys.time() #get time at beginning of trial
              GLMM = glmer(cbind(Yes, Total - Yes) ~ ConditionOfInterest*Difference + (ConditionOfInterest+Difference| ID) + (Difference| StandardValues), 
                           family = binomial(link = "logit"), 
                           data = Psychometric,
                           nAGQ = 1,
                           glmerControl(optimizer = "nloptwrap"))
              RDuration_nloptwrap_nAGQ1 = Sys.time() - TimeStartTrial #get duration of fitting
              RPvalues_nloptwrap_nAGQ1 = summary(GLMM)$coefficients[c(14,16)]
              RCoefficients_nloptwrap_nAGQ1 = summary(GLMM)$coefficients[c(2,4)]
              RAIC_nloptwrap_nAGQ1 = AIC(GLMM)
              RSEs_nloptwrap_nAGQ1 = summary(GLMM)$coefficients[c(6,8)]
            
            
            R = data.frame(iteration = j, 
                 reps = reps,
                 n = n, 
            
                label = c("NelderMead_nAGQ0","NelderMead_nAGQ1","Bobyqa_nAGQ0", "Bobyqa_nAGQ1",
                        "nloptwrap_nAGQ0","nloptwrap_nAGQ1","R: LRT"), 
                AIC = c(RAIC_NelderMead_nAGQ0,RAIC_NelderMead_nAGQ1,RAIC_Bobyqa_nAGQ0,RAIC_Bobyqa_nAGQ1,
                        RAIC_nloptwrap_nAGQ0,RAIC_nloptwrap_nAGQ1,0),
                Pvalues_Accuracy = c(RPvalues_NelderMead_nAGQ0[1],RPvalues_NelderMead_nAGQ1[1],RPvalues_Bobyqa_nAGQ0[1],
                        RPvalues_Bobyqa_nAGQ1[1],RPvalues_nloptwrap_nAGQ0[1],RPvalues_nloptwrap_nAGQ1[1],
                        0),
                Pvalues_Interaction = c(RPvalues_NelderMead_nAGQ0[2],RPvalues_NelderMead_nAGQ1[2],RPvalues_Bobyqa_nAGQ0[2],
                        RPvalues_Bobyqa_nAGQ1[2],RPvalues_nloptwrap_nAGQ0[2],RPvalues_nloptwrap_nAGQ1[2],
                        RPvalues_nloptwrap_nAGQ0_LRT),
                Duration = c(RDuration_NelderMead_nAGQ0,RDuration_NelderMead_nAGQ1,RDuration_Bobyqa_nAGQ0,
                        RDuration_Bobyqa_nAGQ1,RDuration_nloptwrap_nAGQ0,RDuration_nloptwrap_nAGQ1,
                        RDuration_nloptwrap_nAGQ0_LRT),
                Coefficients_Accuracy = c(RCoefficients_NelderMead_nAGQ0[1],RCoefficients_NelderMead_nAGQ1[1],
                                          RCoefficients_Bobyqa_nAGQ0[1], RCoefficients_Bobyqa_nAGQ1[1],
                                          RCoefficients_nloptwrap_nAGQ0[1],RCoefficients_nloptwrap_nAGQ1[1],0),
                Coefficients_Interaction = c(RCoefficients_NelderMead_nAGQ0[2],RCoefficients_NelderMead_nAGQ1[2],
                                          RCoefficients_Bobyqa_nAGQ0[2], RCoefficients_Bobyqa_nAGQ1[2],
                                          RCoefficients_nloptwrap_nAGQ0[2],RCoefficients_nloptwrap_nAGQ1[2],0),
                SEs_Accuracy = c(RSEs_NelderMead_nAGQ0[1],RSEs_NelderMead_nAGQ1[1],
                                          RSEs_Bobyqa_nAGQ0[1], RSEs_Bobyqa_nAGQ1[1],
                                          RSEs_nloptwrap_nAGQ0[1],RSEs_nloptwrap_nAGQ1[1],0),
                SEs_Interaction = c(RSEs_NelderMead_nAGQ0[2],RSEs_NelderMead_nAGQ1[2],
                                          RSEs_Bobyqa_nAGQ0[2], RSEs_Bobyqa_nAGQ1[2],
                                          RSEs_nloptwrap_nAGQ0[2],RSEs_nloptwrap_nAGQ1[2],0),           
            
                ConditionsOfInterest=rep(length(ConditionOfInterest),7), 
                StandardValue1=rep(StandardValues[1],7),
                StandardValue2=rep(StandardValues[2],7), 
                PSE_Difference=rep(PSE_Difference,7), 
                JND_Difference=rep(JND_Difference,7), 
                Multiplicator_PSE_Standard=rep(Multiplicator_PSE_Standard,7), 
                Multiplicator_SD_Standard=rep(Multiplicator_SD_Standard,7),
                SD_ResponseFunction = rep(SD_ResponseFunction,7),
                Mean_Variability_Between = rep(Mean_Variability_Between,7),
                SD_Variability_Between = rep(SD_Variability_Between,7),
                Program = rep("R",7),
                Duration_SimulateDataset=rep(Duration_SimulateDataset,7))

            """
            
            @rget Psychometric Duration_SimulateDataset R
            
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial()) #set up model
            GLMM_Model.optsum.optimizer = :LN_NELDERMEAD #set optimizer to Nelder-Mead
            TimeStartTrial = Dates.now() #save time before fitting
            GLMM = fit!(GLMM_Model, fast=true) #fit model
            JuliaDurationGLMM_NeldMeader_AGP0 = ((Dates.now()) - TimeStartTrial) #save timer after fitting
            JuliaAIC_NeldMeader_AGP0 = aic(GLMM) #save AIC
            JuliaPvalues_NeldMeader_AGP0 = [(coeftable(GLMM)).cols[4][3];(coeftable(GLMM)).cols[4][4]] #save p values. the first one is for a main effect, the second one for the interaction
            JuliaCoefficients_NeldMeader_AGP0 = [(coeftable(GLMM)).cols[1][3];(coeftable(GLMM)).cols[1][4]] #save coefficients. the first one is for a main effect, the second one for the interaction
            JuliaSEs_NeldMeader_AGP0 = [(coeftable(GLMM)).cols[2][3];(coeftable(GLMM)).cols[2][4]] #save standard errors. the first one is for a main effect, the second one for the interaction
            
            #as above, but with fast = false which corresponds to nAGP = 1 in R
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            GLMM_Model.optsum.optimizer = :LN_NELDERMEAD
            TimeStartTrial = Dates.now()
            GLMM = fit!(GLMM_Model, fast=false)
            JuliaDurationGLMM_NeldMeader_AGP1 = ((Dates.now()) - TimeStartTrial)
            JuliaAIC_NeldMeader_AGP1 = aic(GLMM)
            JuliaPvalues_NeldMeader_AGP1 = [(coeftable(GLMM)).cols[4][3];(coeftable(GLMM)).cols[4][4]]
            JuliaCoefficients_NeldMeader_AGP1 = [(coeftable(GLMM)).cols[1][3];(coeftable(GLMM)).cols[1][4]]  
            JuliaSEs_NeldMeader_AGP1 = [(coeftable(GLMM)).cols[2][3];(coeftable(GLMM)).cols[2][4]]
            
            #as above, but with BOBYQA optimizer and fast = false which corresponds to nAGP = 1 in R
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            TimeStartTrial = Dates.now()
            GLMM_Slow = fit!(GLMM_Model, fast=false) #default optimizer is bobyqa
            JuliaDurationGLMM_bobyqa_AGP1 = ((Dates.now()) - TimeStartTrial)
            JuliaAIC_bobyqa_AGP1 = aic(GLMM_Slow)
            JuliaPvalues_bobyqa_AGP1 = [(coeftable(GLMM_Slow)).cols[4][3];(coeftable(GLMM_Slow)).cols[4][4]]
            JuliaCoefficients_bobyqa_AGP1 = [(coeftable(GLMM_Slow)).cols[1][3];(coeftable(GLMM_Slow)).cols[1][4]]
            JuliaSEs_bobyqa_AGP1 = [(coeftable(GLMM_Slow)).cols[2][3];(coeftable(GLMM_Slow)).cols[2][4]]

            #as above, but with BOBYQA optimizer and fast = true which corresponds to nAGP = 0 in R
            GLMM_Model = GeneralizedLinearMixedModel(formula1, Psychometric, Binomial())
            TimeStartTrial = Dates.now()
            GLMM_Fast = fit!(GLMM_Model, fast=true) #default optimizer is bobyqa
            JuliaDurationGLMM_bobyqa_AGP0 = ((Dates.now()) - TimeStartTrial)
            JuliaAIC_bobyqa_AGP0 = aic(GLMM_Fast)
            JuliaPvalues_bobyqa_AGP0 = [(coeftable(GLMM_Fast)).cols[4][3];(coeftable(GLMM_Fast)).cols[4][4]]
            JuliaCoefficients_bobyqa_AGP0 = [(coeftable(GLMM_Fast)).cols[1][3];(coeftable(GLMM_Fast)).cols[1][4]]
            JuliaSEs_bobyqa_AGP0 = [(coeftable(GLMM_Fast)).cols[2][3];(coeftable(GLMM_Fast)).cols[2][4]]

            #fit a null model for Likelihood Ratio Testing
            TimeStartTrial = Dates.now()
            GLMMNull = fit!(GeneralizedLinearMixedModel(formulaNull_Interaction, Psychometric, Binomial()),fast=true)
            Pvalue_LRT_Fast = MixedModels.likelihoodratiotest(GLMM_Fast,GLMMNull).pvalues[1]
            JuliaDurationLRT_Fast = ((Dates.now()) - TimeStartTrial)
            
            TimeStartTrial = Dates.now()
            GLMMNull = fit!(GeneralizedLinearMixedModel(formulaNull_Interaction, Psychometric, Binomial()),fast=false)
            Pvalue_LRT_Slow = MixedModels.likelihoodratiotest(GLMM_Slow,GLMMNull).pvalues[1]
            JuliaDurationLRT_Slow = ((Dates.now()) - TimeStartTrial)
            
            #save all the variables
            Julia = DataFrame(
                        iteration = j,
                        reps = reps,
                        n = n,
                        
                        label = ["JuliaAIC_NeldMeader_AGP0","JuliaAIC_bobyqa_AGP0",
                                 "JuliaAIC_NeldMeader_AGP1","JuliaAIC_bobyqa_AGP1","JuliaLRT_Fast","JuliaLRT_Slow"],
                        AIC = [JuliaAIC_NeldMeader_AGP0,JuliaAIC_bobyqa_AGP0,
                               JuliaAIC_NeldMeader_AGP1,JuliaAIC_bobyqa_AGP1,0,0],
                        Pvalues_Accuracy = [JuliaPvalues_NeldMeader_AGP0[1],JuliaPvalues_bobyqa_AGP0[1],
                                            JuliaPvalues_NeldMeader_AGP1[1],JuliaPvalues_bobyqa_AGP1[1],0,0],
                        Pvalues_Interaction = [JuliaPvalues_NeldMeader_AGP0[2],JuliaPvalues_bobyqa_AGP0[2],
                                               JuliaPvalues_NeldMeader_AGP1[2],JuliaPvalues_bobyqa_AGP1[2],
                                               Pvalue_LRT_Fast,Pvalue_LRT_Slow],
                        Duration = [JuliaDurationGLMM_NeldMeader_AGP0.value/1000,JuliaDurationGLMM_bobyqa_AGP0.value/1000,
                                    JuliaDurationGLMM_NeldMeader_AGP1.value/1000,JuliaDurationGLMM_bobyqa_AGP1.value/1000,
                                    JuliaDurationLRT_Fast.value/1000,JuliaDurationLRT_Slow.value/1000],
                        Coefficients_Accuracy = [JuliaCoefficients_NeldMeader_AGP0[1],JuliaCoefficients_NeldMeader_AGP1[1],
                                                 JuliaCoefficients_bobyqa_AGP1[1],JuliaCoefficients_bobyqa_AGP0[1],0,0],
                        Coefficients_Interaction = [JuliaCoefficients_NeldMeader_AGP0[2],JuliaCoefficients_NeldMeader_AGP1[2],
                                                 JuliaCoefficients_bobyqa_AGP1[2],JuliaCoefficients_bobyqa_AGP0[2],0,0],
                        SEs_Accuracy = [JuliaSEs_NeldMeader_AGP0[1],JuliaSEs_NeldMeader_AGP1[1],
                                                 JuliaSEs_bobyqa_AGP1[1],JuliaSEs_bobyqa_AGP0[1],0,0],
                        SEs_Interaction = [JuliaSEs_NeldMeader_AGP0[2],JuliaSEs_NeldMeader_AGP1[2],
                                                 JuliaSEs_bobyqa_AGP1[2],JuliaSEs_bobyqa_AGP0[2],0,0],                
                
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard,
                        SD_ResponseFunction = SD_ResponseFunction,
                        Mean_Variability_Between = Mean_Variability_Between,
                        SD_Variability_Between = SD_Variability_Between,
                        Program = "Julia",
                        Duration_SimulateDataset=Duration_SimulateDataset)
            
            if CurrentRunthrough == 1 && j == 1 && reps == Range_reps[1] && n == Range_Participants[1]
            
                global PowerfulDataframe = append!(R,Julia)
                
            else
                
                PowerfulDataframe = append!(PowerfulDataframe,R)
                PowerfulDataframe = append!(PowerfulDataframe,Julia)
            end
        end
    end
end

#write all variables to CSV
CSV.write(join(["ComparisonMethodsNoEffect.csv"]),PowerfulDataframe)

Starting runthrough N° 1 out of 24

┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.00285609 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.00212775 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\

┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.00280686 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\

This runthrough took 1324149 milliseconds sStarting runthrough N° 2 out of 24

┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.00399376 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.00330052 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\

...

┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.0317286 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.00461424 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g

This runthrough took 6207937 milliseconds sStarting runthrough N° 24 out of 24

┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.00413862 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.00123358 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\

┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   unable to evaluate scaled gradient
│ Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge: degenerate  Hessian with 1 negative eigenvalues
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: Warning in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
│   Model failed to converge with max|grad| = 0.0404886 (tol = 0.001, component 1)
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: boundary (singular) fit: see ?isSingular
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl

"ComparisonMethodsNoEffect.csv"